In [1]:
import numpy as np

In [3]:
# Funciones de activación
from tensorflow.keras.activations import elu, exponential, gelu, hard_sigmoid, linear, relu, selu, sigmoid, softmax, softplus, softsign, swish, tanh

2023-09-07 23:27:14.140243: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-07 23:27:14.251540: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-07 23:27:14.253729: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 23:27:15.491927: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# Optimizadores
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam, Adagrad

In [5]:
# Funciones de perdida
from tensorflow.keras.losses import binary_crossentropy, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error

In [2]:
class Neurona():
    def __init__(self, n_features, func_activ):
        self.pesos = np.random.rand(n_features)
        self.bias = np.random.rand()
        self.func_activ = func_activ
    
    def CalcEntradaNeta(self, entrada):
        return np.dot(entrada, self.pesos) + self.bias
    
    def Salida(self, entrada):
        return self.func_activ(self.CalcEntradaNeta(entrada))
    
    # FuncPerdida esta más relacionado con el proceso de entrenamiento 
    # pero lo estamos englobando en la neurona
    def FuncPerdida(self, func_perdida):
        self.func_perdida = func_perdida
    
    def CalcError(self, entrada, y_real):
        return self.func_perdida([self.Salida(entrada)], y_real)

Internet

In [4]:
import numpy as np
from sklearn.datasets import load_digits

In [5]:
X, y = load_digits(return_X_y=True)
print(X.shape)
print(y.shape)

(1797, 64)
(1797,)


In [6]:
import random
# Preferí la implementación de random.sample porque me permite generar n valores random sin repetir
classes = random.sample(range(y.min(), y.max()), 2)
print(f"Se va a entrenar para reconocer las clases: {classes}")

Se va a entrenar para reconocer las clases: [4, 7]


In [7]:
#filtered_x = np.ndarray(shape=(0, X.shape[1])) Mala práctica según la documentación
filtered_x = np.empty(shape=(0, X.shape[1]))
filtered_y = np.empty(shape=0)

for clas in classes:
    mask_class = np.ma.masked_where(y == clas, y)
    filtered_x = np.concatenate((filtered_x, X[mask_class.mask]))
    filtered_y = np.concatenate((filtered_y, y[mask_class.mask]))

#Validando filtro
print(f"¿El número de ejemplos en x filtrado ({filtered_x.shape[0]}) \
es igual al de y filtado? \
{filtered_x.shape[0] == filtered_y.shape[0]}")
print(f"Las clases filtradas son: {np.unique(filtered_y)}")


¿El número de ejemplos en x filtrado (360) es igual al de y filtado? True
Las clases filtradas son: [4. 7.]


In [8]:
from sklearn.preprocessing import MinMaxScaler
# Normalizamos los datos para tener un rango 0-1 reemplazando 0-16
scaler = MinMaxScaler().fit(filtered_x)
normalized_x = scaler.transform(filtered_x)
print(f"El rango de valores por pixel se cambio a {normalized_x.min()} - {normalized_x.max()}")


El rango de valores por pixel se cambio a 0.0 - 1.0


In [9]:
from sklearn.model_selection import train_test_split
# Generamos dos datasets (train 70% y test 30%) 
x_train, x_test, y_train, y_test = train_test_split(normalized_x, filtered_y, test_size=0.30)

# Validando distribución de nuestras clases en train
for clas in classes:
    print(f"Cantidad de {clas} = {np.count_nonzero(y_train == clas)}")


Cantidad de 4 = 131
Cantidad de 7 = 121


In [13]:
import numpy as np

# Activation Function
def sigmoid(w_sum):
    return 1/(1+np.exp(-w_sum))

# Get Prediction
def predict(features, weights, bias):
    return sigmoid(np.dot(features, weights) + bias)

# Loss Function
def cross_entropy(target, pred):
    return -(target*np.log10(pred)+(1-target)*(np.log10(1-pred)))

# Update Weights
def gradient_descent(x, y, weights, bias, learnrate, pred):
    new_weights = []
    bias += learnrate*(y-pred)

    for w,xi in zip(weights,x):
        new_weight = w + learnrate*(y-pred)*xi
        new_weights.append(new_weight) 
    return new_weights, bias

# Data
#features = np.array(([0.1,0.5,0.2],[0.2,0.3,0.1],[0.7,0.4,0.2],[0.1,0.4,0.3]))
#targets = np.array([0,1,0,1])
features = x_train
targets = y_train

epochs = 10
learnrate = 0.1
    
errors = []
#weights = np.array([0.4, 0.2, 0.6])
weights = np.random.rand(x_train.shape[1])
bias = 0.5

new_weights = []

for e in range(epochs):
    for x, y in zip(features, targets):
        pred = predict(x, weights, bias)
        error = cross_entropy(y, pred)
        weights, bias = gradient_descent(x, y, weights, bias, learnrate, pred)
    
    # Printing out the log-loss error on the training set
    out = predict(features, weights, bias)
    loss = np.mean(cross_entropy(targets, out))
    errors.append(loss)
    print("\n========== Epoch", e,"==========")
    print("Average loss: ", loss)

/tmp/ipykernel_22800/842422510.py:13: RuntimeWarning: divide by zero encountered in log10
  return -(target*np.log10(pred)+(1-target)*(np.log10(1-pred)))



========== Epoch 0 ==========
Average loss:  -inf

========== Epoch 1 ==========
Average loss:  -inf

========== Epoch 2 ==========
Average loss:  -inf

========== Epoch 3 ==========
Average loss:  -inf

========== Epoch 4 ==========
Average loss:  -inf

========== Epoch 5 ==========
Average loss:  -inf

========== Epoch 6 ==========
Average loss:  -inf

========== Epoch 7 ==========
Average loss:  -inf

========== Epoch 8 ==========
Average loss:  -inf

========== Epoch 9 ==========
Average loss:  -inf
